In [ ]:
source('datamaker.R')

In [3]:
library(limma)
library(edgeR)
library(ashr)

# the baseline
VL_ash <- function(input) {
    sebetahat <- input$sebetahat.voom
    betahat <- input$betahat.voom
    df <- input$df.voom
    fit <- ash(betahat, sebetahat, df = df, method = "fdr", mixcompdist = NULL)
    return(fit)
}

# the proposed method based on solving the EBTM problem
VL_eBayes_ash <- function(input, alpha) {
    dgecounts <- calcNormFactors(DGEList(counts = input$counts, group = input$condition))
    design <- model.matrix(~input$condition)
    v <- voom(dgecounts, design, plot = FALSE)
    lim <- lmFit(v)
    lim <- eBayes(lim)
    betahat <- lim$coefficients[, 2]
    sebetahat <- lim$stdev.unscaled[, 2] * sqrt(lim$s2.post)
    df <- lim$df.total[1]
    fit <- ash(betahat, sebetahat, df = df, alpha = alpha)
    return(fit)
}

# the ad hoc approach
VL_pval2se_ash <- function(input) {
    lim <- lmFit(input$v)
    lim <- eBayes(lim)
    betahat <- lim$coefficients[, 2]
    pval <- lim$p.value[, 2]
    zscore <- qnorm(1 - pval/2)
    sebetahat <- abs(betahat/zscore)
    fit <- ash(betahat, sebetahat, method = "fdr")
    return(fit)
}

In [4]:
ngene <- 10000
nsample <- c(2, 4, 10)
scale <- c(0.25, 1, 3)/2
nrep <- 50
set.seed(5472)

In [5]:
priors <- list(
    list(
        name = "spiky",
        betapi = c(0.4, 0.2, 0.2, 0.2),
        betamu = c(0, 0, 0, 0),
        betasd = c(0.25, 0.5, 1, 2)),
    list(
        name = "near_normal",
        betapi = c(2/3, 1/3),
        betamu = c(0, 0),
        betasd = c(1, 2)),
    list(
        name = "flat_top",
        betapi = rep(1/7, 7), 
        betamu = c(-1.5, -1, -0.5, 0, 0.5, 1, 1.5),
        betasd = rep(0.5, 7)),
    list(
        name = "big_normal",
        betapi = c(1),
        betamu = c(0),
        betasd = c(4)), 
    list(
        name = "bimodal",
        betapi = c(0.5, 0.5),
        betamu = c(-2, 2),
        betasd = c(1, 1))
)

In [ ]:
df <- data.frame()
for (i in 1:length(nsample)) {
    for (j in 1:length(priors)) {
        for (k in 1:nrep) {
            prior <- priors[[j]]
            data <- datamaker(list(
                ngene = ngene,
                nsamp = nsample[i],
                pi0 = "random", 
                betaargs = list(
                    betaapi = prior$betapi,
                    betamu = prior$betamu,
                    betasd = prior$betasd/scale[i]),
                breaksample = TRUE)
            )
            ash_fit <- VL_ash(data$input)
            eBayes_fit <- VL_eBayes_ash(data$input, 0)
            eBayes_alph1_fit <- VL_eBayes_ash(data$input, 1)
            pval2se_fit <- VL_pval2se_ash(data$input)
            beta <- data$meta$beta
            ash_rmse <- sqrt(mean((beta - ash_fit$result$PosteriorMean)^2))
            eBayes_rmse <- sqrt(mean((beta - eBayes_fit$result$PosteriorMean)^2))
            eBayes_alpha1_rmse <- sqrt(mean((beta - eBayes_alph1_fit$result$PosteriorMean)^2))
            pval2se_rmse <- sqrt(mean((beta - pval2se_fit$result$PosteriorMean)^2))
            result <- data.frame(
                method = c("VL+ash", "VL+eBayes+ash", "VL+eBayes+ash.alpha=1", "VL+pval2se+ash"),
                prior = rep(prior$name, 4),
                nsample = rep(nsample[i], 4),
                pi0 = rep(data$meta$pi0, 4),
                pi0_est = c(
                    ash_fit$fitted_g$pi[1],
                    eBayes_fit$fitted_g$pi[1], 
                    eBayes_alph1_fit$fitted_g$pi[1],
                    pval2se_fit$fitted_g$pi[1]), 
                rrmse = c(
                    1,
                    eBayes_rmse/ash_rmse,
                    eBayes_alpha1_rmse/ash_rmse,
                    pval2se_rmse/ash_rmse)
            )
            df <- rbind(df, result)
        }
        cat(i, j, "\n")
        flush.console()
    }
}

In [7]:
write.csv(df, "result.csv", row.names = FALSE)